In [1]:
import sys,os
sys.path.append("..")
import django
django.setup()
import datetime
import pandas as pd
from tw_stock_class.finlab.crawler import (
    # 爬蟲們
    crawl_price,
    crawl_bargin,
    crawl_pe,
    crawl_benchmark,
    crawl_monthly_report,
    crawl_twse_divide_ratio,
    crawl_otc_divide_ratio,
    crawl_twse_cap_reduction,
    crawl_otc_cap_reduction,

)
import logging
from django.conf import settings

logging.basicConfig(level=logging.INFO)
BASE_DIR='/Users/benbilly3/finlab/finlab_data_center/finlab_data_center/tw_stock_class'

# Download today data

In [6]:
def git_init():
    user_name = os.getenv("GITLAB_NAME", settings.CONFIG_DATA.get("GITLAB_NAME"))
    user_password = os.getenv("GITLAB_PASSWORD", settings.CONFIG_DATA.get("GITLAB_PASSWORD"))
    mail = os.getenv("GMAIL", settings.CONFIG_DATA.get("GMAIL"))
    os.chdir(BASE_DIR)
    os.system(f"git config --local user.email {mail}")
    os.system(f"git config --local user.name {user_name}")
 

def git_commit():
    git_init()
    now = datetime.datetime.now()
    now = str(now.strftime("%Y-%m-%d %H:%M"))
    os.system("git add data/")
    os.system("git commit -m '%s update data'" % now)
    os.system("git push -u origin master")
    info = "Finish!git push successfully"
    return info

In [7]:
class TwStockClassPickler:
    crawlers_list = ['crawl_price', 'crawl_bargin', 'crawl_pe', 'crawl_benchmark', 'crawl_monthly_report']
    crawlers_once_list = ['crawl_twse_divide_ratio', 'crawl_otc_divide_ratio', 'crawl_twse_cap_reduction',
                          'crawl_otc_cap_reduction']

    def download_single(self, func: str, dt=False, once=False):
        if dt is False:
            dt = datetime.datetime.now()
        dir_name = func[func.index('_') + 1:]

        if func != 'crawl_monthly_report':
            file_name = dt.strftime('%Y%m%d')
        else:
            file_name = dt.strftime('%Y%m')

        if once:
            df = eval(func)()
            if df.empty is False:
                df.to_pickle(f'{BASE_DIR}/data/{dir_name}/{dir_name}.pickle')
        else:
            df = eval(func)(dt)
            if df.empty is False:
                df.to_pickle(f'{BASE_DIR}/data/{dir_name}/{file_name}.pickle')
        return df

    def download_all(self, dt=False):
        git_init()
        os.system("git pull origin master")
        if dt is False:
            dt = datetime.date.today()
            dt = datetime.datetime(dt.year, dt.month, dt.day)
        for crawler_name in self.crawlers_list:
            self.download_single(crawler_name, dt)

        for crawler_name in self.crawlers_once_list:
            self.download_single(crawler_name, once=True)
        info = 'Finish!download class data and divide pickle file'
        return info
# TwStockClassPickler().download_all()

In [9]:
git_commit()

'Finish!git push successfully'

In [ ]:
# import urllib.request
# import pickle
# import requests

# def crawl_gitlab_backup(target:str,date=None):
#     crawlers_once_list=['twse_divide_ratio','otc_divide_ratio','twse_cap_reduction','otc_cap_reduction']
#     headers = {
#                     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko)'
#                                   ' Chrome/39.0.2171.95 Safari/537.36'
#     }
    
#     if date is not None:
#         date_str=date.strftime('%Y%m%d')
#         if target== 'monthly_report':
#             date_str=date.strftime('%Y%m')

#     if target in crawlers_once_list:
#         url=f'https://gitlab.com/finlab_company_class/tw_stock/-/raw/master/data/{target}/{target}.pickle?inline=false'
#     else:
#         url=f'https://gitlab.com/finlab_company_class/tw_stock/-/raw/master/data/{target}/{date_str}.pickle?inline=false'
#     try:
#         req = urllib.request.Request(url=url, headers=headers)
#         df = pickle.load(urllib.request.urlopen(req))
#         return df
#     except:
#         return None
    

# date=date=datetime.datetime(2020, 10, 5, 0, 0)
# # crawl_gitlab_backup('twse_divide_ratio')
# # crawl_gitlab_backup('price',date)
# crawl_gitlab_backup('monthly_report',date)


In [8]:
import time
c=TwStockClassPickler()
date_list=pd.date_range(start='21/12/2020', end='30/12/2020')
for test_dt in date_list:
    c.download_all(test_dt)
    time.sleep(5)

--- Logging error ---
Traceback (most recent call last):
  File "/Users/benbilly3/opt/anaconda3/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/benbilly3/opt/anaconda3/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/benbilly3/opt/anaconda3/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/benbilly3/opt/anaconda3/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/benbilly3/opt/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/benbilly3/opt/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/benbilly3/finlab/finlab_data_center/venv/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in 

INFO:tw_stock_class.finlab.crawler:https://mops.twse.com.tw/nas/t21/otc/t21sc03_109_11.html
/Users/benbilly3/finlab/finlab_data_center/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mops.twse.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
INFO:tw_stock_class.finlab.crawler:https://mops.twse.com.tw/nas/t21/sii/t21sc03_109_11.html
/Users/benbilly3/finlab/finlab_data_center/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mops.twse.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
INFO:tw_stock_class.finlab.crawler:https://mops.twse.com.tw/nas/t21/otc/t21sc03_109_11.html
/Users/benbi

In [4]:
mail = os.getenv('GMAIL', settings.CONFIG_DATA.get("GMAIL"))
f"git config --global user.email {mail}"

'git config --global user.email finlab.company@gmail.com'

In [4]:
pd.date_range(start='21/12/2020', end='30/12/2020')

DatetimeIndex(['2020-12-21', '2020-12-22', '2020-12-23', '2020-12-24',
               '2020-12-25', '2020-12-26', '2020-12-27', '2020-12-28',
               '2020-12-29', '2020-12-30'],
              dtype='datetime64[ns]', freq='D')